In [0]:
##install pytorch in colab
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

In [1]:
##mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
import json
import pandas as pd
import torch
import time
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn.init as init

In [0]:
##generate sphere-shell binary data
def data_sphere_shell(num_samples,points_dim,radius_of_sphere, radius_of_shell, thickness_of_shell):

    def get_sphere_shell_label(v):           
        return 1 if np.sqrt(np.sum(np.square(v))) <= radius_of_sphere else 0

    points = np.zeros([num_samples, points_dim])
    labels = np.zeros([num_samples,1]).astype(int)
    
    # Generate positive points inside the circle.
    for i in range(num_samples // 2):
        r = random.uniform(0, radius_of_sphere)
        angle = np.zeros(points_dim-1)
        for j in range(points_dim-1):
            angle[j] = random.uniform(0, 2 * np.pi)
        
        x = np.zeros(points_dim)
        for j in range(points_dim-1):
            x[j] = r * np.cos(angle[j])
            r = r * np.sin(angle[j])
        x[points_dim-1] = r
        labels[i] = get_sphere_shell_label(x)
        points[i] = x

    # Generate negative points outside the circle.
    for i in range(num_samples // 2, num_samples):
        r = random.uniform(radius_of_shell-(thickness_of_shell/2), radius_of_shell+(thickness_of_shell/2))
        angle = np.zeros(points_dim-1)
        for j in range(points_dim-1):
            angle[j] = random.uniform(0, 2 * np.pi)
        
        x = np.zeros(points_dim)
        for j in range(points_dim-1):
            x[j] = r * np.cos(angle[j])
            r = r * np.sin(angle[j])
        x[points_dim-1] = r
        labels[i] = get_sphere_shell_label(x)
        points[i] = x
        
    return points, labels
    
points_dim = 6
num_samples = 500
radius_of_sphere = 1
radius_of_shell = 2.4
thickness_of_shell = 1
points, labels = data_sphere_shell(num_samples,points_dim,radius_of_sphere, radius_of_shell, thickness_of_shell)

#path to csv file to store generated data
csv_path = '/content/drive/My Drive/DL_exp/data/train_sp'+ str(radius_of_sphere) + '_sh'+ str(radius_of_shell) + '_t' + str(thickness_of_shell)+ '_' + str(points_dim) + 'dim.csv'     
with open(csv_path, 'w') as writeFile:
    writer = csv.writer(writeFile)
    header = ["c"+str(i) for i in range(points_dim)]
    header.append("label")
    writer.writerow(header)
    writer.writerows(np.hstack((points,labels)))

In [0]:
#dataset class
class load_dataset(Dataset):
    def __init__(self, data_path):
        self.samples = pd.read_csv(data_path).values

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx,:-1] ,self.samples[idx,-1]
      
###Model

class MultilayerPerceptron(torch.nn.Module):

    def __init__(self):
        super(MultilayerPerceptron, self).__init__()
        
        self.hidden = torch.nn.ModuleList()
        self.hidden.append(torch.nn.Linear(num_features, hidden_nodes_list[0]))
        for k in range(num_hidden_layes-1):
            self.hidden.append(torch.nn.Linear(hidden_nodes_list[k], hidden_nodes_list[k+1]))
             
        ### Output layer
        self.hidden.append(torch.nn.Linear(hidden_nodes_list[num_hidden_layes-1], num_classes))
        
    def forward(self, x):
        out = x
        for layer in self.hidden[:-1]:
          out = layer(out)
          out = F.relu(out)

        logits = self.hidden[-1](out)
        probas = F.log_softmax(logits, dim=1)
        return logits, probas
      
#weight initialization function
def init_weights(m):
  if isinstance(m, torch.nn.Linear):
    if initialisation_method=='xavier':
      init.xavier_uniform_(m.weight)
    if initialisation_method=='he':
      init.kaiming_uniform_(m.weight)
    m.bias.data.fill_(0.01)
    
#compute accuracy function
def compute_accuracy(net, data_loader):
    net.eval()
    correct_pred, num_examples = 0, 0
    with torch.no_grad():
        for features, targets in data_loader:
            features = features.float().to(device)
            targets = targets.long().to(device)
            logits, probas = net(features)
            cost = F.cross_entropy(logits, targets)
            _, predicted_labels = torch.max(probas, 1)
            num_examples += targets.size(0)
            correct_pred += (predicted_labels == targets).sum()
        return cost,correct_pred.float()/num_examples * 100

      

In [0]:
# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#data
points_dim = 6
radius_of_sphere = 1
radius_of_shell = 2.4
thickness_of_shell = 1
csv_path = '/content/drive/My Drive/DL_exp/data/train_sp'+ str(radius_of_sphere) + '_sh'+ str(radius_of_shell) + '_t' + str(thickness_of_shell)+ '_' + str(points_dim) + 'dim.csv'     

dataset = load_dataset(csv_path)
size = len(dataset)


#architecture
num_features  = next(iter(dataset))[0].shape[0]        # Input data dimention
hidden_nodes_list   = [4]                              # List of number of nodes at each hidden layer
num_hidden_layes = len(hidden_nodes_list)              # The number of hidden layers
num_classes   = 2                                      # The number of output classes. In this case, 0 and 1

# 'xavier' : Xavier Initialisation
# 'he' : He Initialisation
initialisation_method = 'xavier'

# 'sgd' : SGD (lr) 
# 'sgdwm' : SGD with Momentum (lr, momentum)
# 'adagrad' : AdaGrad
# 'adam' : Adam
# 'ngd' : Natural gradient descent
# 'l1' : L1 Regularisation
# 'l2' : L2 Regularisation
# 'pathnorm' : PathNorm Regularisation
# 'spectralnorm' : Spectral norm Regularisation
optimisation_method = 'sgdwm'  

# Hyperparameters
random_seed = 400
learning_rate = 0.03
num_epochs = 100
batch_size = size

##load dataset
train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

In [6]:
### training of a model
torch.manual_seed(random_seed)
model = MultilayerPerceptron()
model.apply(init_weights)
model = model.to(device)

if optimisation_method=='sgd' or optimisation_method=='ngd':
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
if optimisation_method=='sgdwm':
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
if optimisation_method=='adagrad':
  optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate)
if optimisation_method=='adam':
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, betas=(0.9, 0.999), eps=1e-08)
  
  
start_time = time.time()
epoch = 0

with torch.set_grad_enabled(False):
  cost,best_acc = compute_accuracy(model, train_dataloader)
  print('Epoch: %03d | Accuracy: %.2f%% | Cost: %.4f' % (epoch,best_acc,cost))
  
# for epoch in range(num_epochs):
count = 1
prev_acc=best_acc
best_epoch = epoch
best_cost = cost

while True:
    model.train()
    for batch_idx, (features, targets) in enumerate(train_dataloader):
        
        features = features.float().to(device)
        targets = targets.long().to(device)
            
        ### FORWARD AND BACK PROP
        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        
        ### UPDATE MODEL PARAMETERS
        optimizer.step()
        

    with torch.set_grad_enabled(False):
        cost,acc = compute_accuracy(model, train_dataloader)
        print('Epoch: %03d | Accuracy: %.2f%% | Cost: %.4f' % (epoch,acc,cost))
        
    epoch+=1
    if prev_acc==acc:
      count+=1
    else:
      prev_acc=acc
      count=1
    if (epoch>50 and best_acc-acc>=5) or count==20 or epoch==400:
      break
    if acc>best_acc:
      best_acc=acc
      best_epoch = epoch
      best_cost = cost
    
#     print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

Epoch: 000 | Accuracy: 53.40% | Cost: 0.9229
Epoch: 000 | Accuracy: 52.40% | Cost: 0.9135
Epoch: 001 | Accuracy: 49.60% | Cost: 0.8964
Epoch: 002 | Accuracy: 46.20% | Cost: 0.8738
Epoch: 003 | Accuracy: 41.00% | Cost: 0.8479
Epoch: 004 | Accuracy: 36.00% | Cost: 0.8211
Epoch: 005 | Accuracy: 30.00% | Cost: 0.7952
Epoch: 006 | Accuracy: 25.60% | Cost: 0.7718
Epoch: 007 | Accuracy: 30.40% | Cost: 0.7515
Epoch: 008 | Accuracy: 33.00% | Cost: 0.7345
Epoch: 009 | Accuracy: 36.00% | Cost: 0.7208
Epoch: 010 | Accuracy: 38.20% | Cost: 0.7096
Epoch: 011 | Accuracy: 39.60% | Cost: 0.7004
Epoch: 012 | Accuracy: 40.00% | Cost: 0.6925
Epoch: 013 | Accuracy: 40.20% | Cost: 0.6854
Epoch: 014 | Accuracy: 40.40% | Cost: 0.6789
Epoch: 015 | Accuracy: 41.40% | Cost: 0.6725
Epoch: 016 | Accuracy: 41.60% | Cost: 0.6662
Epoch: 017 | Accuracy: 41.60% | Cost: 0.6599
Epoch: 018 | Accuracy: 41.40% | Cost: 0.6535
Epoch: 019 | Accuracy: 41.20% | Cost: 0.6471
Epoch: 020 | Accuracy: 40.60% | Cost: 0.6407
Epoch: 021

In [27]:
### training of multiple models and save the result in JSON file
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

start_time = time.time()
radius_of_sphere = 1
radius_of_shell = 2.4
thickness_of_shell = 1

dimentions = [2,3,4,5,6]          #list of dimention of data
for dim in dimentions:
  csv_path = '/content/drive/My Drive/DL_exp/data/train_sp'+ str(radius_of_sphere) + '_sh'+ str(radius_of_shell) + '_t' + str(thickness_of_shell)+ '_' + str(dim) + 'dim.csv'
  dataset = load_dataset(csv_path)
  size = len(dataset)
  train_dataloader = DataLoader(dataset, batch_size=size, shuffle=True)


  tmp_h = [dim,dim+1,dim+3,dim+5,dim+7,dim+9]             #list of number of nodes in hidden layer
  tmp_i = [100,200,300,400,500,600,700,800,900,1000]      #list of random seed for weight initialization

  for t_h in tmp_h:
    for t_i in tmp_i:

      #architecture
      num_features  = next(iter(dataset))[0].shape[0]        # Input data dimention
      hidden_nodes_list   = [t_h]                              # List of number of nodes at each hidden layer
      num_hidden_layes = len(hidden_nodes_list)              # The number of hidden layers
      num_classes   = 2                                      # The number of output classes. In this case, 0 and 1

      initialisation_method = 'xavier'
      optimisation_method = 'sgdwm'  

      # Hyperparameters
      random_seed = t_i
      learning_rate = 0.05
            
      torch.manual_seed(random_seed)
      model = MultilayerPerceptron()
      model.apply(init_weights)
      model = model.to(device)

      optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
      
      epoch = 0

      with torch.set_grad_enabled(False):
        cost,best_acc = compute_accuracy(model, train_dataloader)

      # for epoch in range(num_epochs):
      count = 1
      prev_acc=best_acc
      best_epoch = epoch
      best_cost = cost

      while True:
          model.train()
          for batch_idx, (features, targets) in enumerate(train_dataloader):

              features = features.float().to(device)
              targets = targets.long().to(device)

              ### FORWARD AND BACK PROP
              logits, probas = model(features)
              cost = F.cross_entropy(logits, targets)
              optimizer.zero_grad()

              cost.backward()

              ### UPDATE MODEL PARAMETERS
              optimizer.step()


          with torch.set_grad_enabled(False):
              cost,acc = compute_accuracy(model, train_dataloader)
              
          epoch+=1
          if prev_acc==acc:
            count+=1
          else:
            prev_acc=acc
            count=1
          if (epoch>50 and best_acc-acc>=5) or count==20 or epoch==400:
            break
          if acc>best_acc:
            best_acc=acc
            best_epoch = epoch
            best_cost = cost
 
      print('Epoch: %03d | Accuracy: %.2f%% | Cost: %.4f' % (epoch,best_acc,cost))
  
      with open('/content/drive/My Drive/DL_exp/result/result.json', mode='r') as readjson:
        try: 
          result = json.loads(readjson.read())
        except ValueError: 
          result = []  
      
        
      with open('/content/drive/My Drive/DL_exp/result/result.json', mode='w') as feedjson:
        entry = {'sphere_radius': '1', 'shell_radius': '2.4', 'thickness' : '1', 'dimension':str(dim), 'n_nodes':str(t_h), 'random_seed':str(t_i), 'epoch':str(best_epoch), 'accuracy':str(best_acc.cpu().numpy()), 'cost':str(best_cost.cpu().numpy())}
        result.append(entry)
        json.dump(result, feedjson)


Epoch: 154 | Accuracy: 89.40% | Cost: 0.2570
Epoch: 150 | Accuracy: 89.40% | Cost: 0.2797
Epoch: 051 | Accuracy: 68.60% | Cost: 0.6064
Epoch: 141 | Accuracy: 69.00% | Cost: 0.5514
Epoch: 197 | Accuracy: 85.20% | Cost: 0.3508
Epoch: 178 | Accuracy: 87.80% | Cost: 0.2916
Epoch: 209 | Accuracy: 85.60% | Cost: 0.3470
Epoch: 187 | Accuracy: 87.80% | Cost: 0.3184
Epoch: 138 | Accuracy: 71.20% | Cost: 0.5306
Epoch: 215 | Accuracy: 89.80% | Cost: 0.2563
Epoch: 073 | Accuracy: 100.00% | Cost: 0.0698
Epoch: 112 | Accuracy: 100.00% | Cost: 0.0614
Epoch: 150 | Accuracy: 100.00% | Cost: 0.0581
Epoch: 073 | Accuracy: 100.00% | Cost: 0.0796
Epoch: 124 | Accuracy: 100.00% | Cost: 0.0664
Epoch: 102 | Accuracy: 100.00% | Cost: 0.0577
Epoch: 091 | Accuracy: 100.00% | Cost: 0.0656
Epoch: 069 | Accuracy: 100.00% | Cost: 0.0673
Epoch: 072 | Accuracy: 100.00% | Cost: 0.0629
Epoch: 136 | Accuracy: 100.00% | Cost: 0.0517
Epoch: 071 | Accuracy: 100.00% | Cost: 0.0717
Epoch: 084 | Accuracy: 100.00% | Cost: 0.049

In [0]:
with open('/content/drive/My Drive/DL_exp/result/result.json', mode='r') as readjson:
  result = json.loads(readjson.read())
  len(result)

In [0]:
keys = result[0].keys()
with open('/content/drive/My Drive/DL_exp/result/result.csv', 'w') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(result)